In [1]:
import  cv2
import numpy as np
from keras.models import load_model, Sequential
from keras.layers import Dense, BatchNormalization, Dropout

from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace

#For local CPU usage:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

#face detection and extraction
from mtcnn.mtcnn import MTCNN
from PIL import Image
from numpy import asarray

Using TensorFlow backend.


In [2]:
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224

EMOTION_DICT = {1:"ANGRY", 2:"DISGUST", 3:"FEAR", 4:"HAPPY", 5:"NEUTRAL", 6:"SAD", 7:"SURPRISE"}

### Define functions and model from training, necessary for prediction

In [3]:
detector = MTCNN()

def detect_faces(image):
    faces = detector.detect_faces(image)
    return np.array(faces)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [4]:
def extract_face_from_image_xywh(image, required_size=(IMAGE_HEIGHT, IMAGE_WIDTH)):
    faces = detect_faces(image)
    face_array = []
    
    for elem in faces:
        # extract the bounding box from the requested face
        x1, y1, width, height = elem['box']
        face_array.append([x1, y1, width, height])
        
    return face_array

In [5]:
def model_top(input_shape):
    model = Sequential()
        
    model.add(Dense(512, activation='relu', input_dim = input_shape))
    model.add(Dropout(0.1))
    
    model.add(Dense(256, activation='relu'))
    
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    
    model.add(Dense(64, activation='relu'))
    model.add(Dense(output_dim = 7, activation='softmax')) 
    
    return model

In [6]:
model_top = model_top(2048)

c:\users\tobias\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=7)`
  del sys.path[0]


In [7]:
model_VGG = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
model_top.load_weights("../training/model_checkpoints/model_top.h5")

### Create the application

In [8]:
### faces = extract_face_from_image(gray) 

def return_prediction(path):
    #converting image to gray scale and save it
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(path, gray)
    
    #detect face in image, crop it then resize it then save it<
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        face_clip = img[y:y+h, x:x+w]
        cv2.imwrite(path, cv2.resize(face_clip, (350, 350)))
    
    #read the processed image then make prediction and display the result
    read_image = cv2.imread(path)
    read_image = read_image.reshape(1, read_image.shape[0], read_image.shape[1], read_image.shape[2])
    read_image_final = read_image/255.0
    VGG_Pred = model_VGG.predict(read_image_final)
    VGG_Pred = VGG_Pred.reshape(1, VGG_Pred.shape[1]*VGG_Pred.shape[2]*VGG_Pred.shape[3])
    top_pred = model_top.predict(VGG_Pred)
    emotion_label = top_pred[0].argmax() + 1
    return EMOTION_DICT[emotion_label]

In [9]:
#def rerun(text, cap):
#    while(True):
#        ret, img = cap.read()
#        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#        font = cv2.FONT_HERSHEY_SIMPLEX
#        cv2.putText(img, "Last Emotion was "+str(text), (95,30), font, 1.0, (255, 0, 0), 2, cv2.LINE_AA)
#        
#        cv2.putText(img, "Press SPACE: FOR EMOTION", (5,470), font, 0.7, (255, 0, 0), 2, cv2.LINE_AA)
#    
#        cv2.putText(img, "Hold Q: To Quit", (460,470), font, 0.7, (255, 0, 0), 2, cv2.LINE_AA)
#    
#        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
#        for x,y,w,h in faces:
#            cv2.rectangle(img, (x,y), (x+w, y+h), (255, 0, 0), 2)
#        
#        cv2.imshow("Image", img)
#        
#        if cv2.waitKey(1) == ord(' '):
#            cv2.imwrite("test.jpg", img)
#            text = return_prediction("test.jpg")
#            first_run(text, cap)
#            break
#            
#        if cv2.waitKey(1) == ord('q'):
#            cap.release()
#            cv2.destroyAllWindows()
#            break

In [10]:
## ## face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 

cap = cv2.VideoCapture(0)

def run_app(text, cap):
    while(True):
        ret, img = cap.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, "Last Emotion was "+str(text), (95,30), font, 1.0, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(img, "Press SPACE: FOR EMOTION", (5,470), font, 0.7, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(img, "Hold Q: To Quit", (460,470), font, 0.7, (255, 0, 0), 2, cv2.LINE_AA)
        
        
        face_array = extract_face_from_image_xywh(gray)
        for elem in face_array:
            x,y,w,h = elem
            cv2.rectangle(img, (x,y), (x+w, y+h), (255, 0, 0), 2)
        
        cv2.imshow("Emotion Recognition - MasterThesis", img)
        
        if cv2.waitKey(1) == ord(' '):
            cv2.imwrite("test.jpg", img)
            ### text = return_prediction("test.jpg")
            ### run_app(text, cap)
            run_app("No Emotion", cap)
            break
            
        if cv2.waitKey(1) == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break

In [11]:
run_app("None", cap)